# Projeto - Natural Language Processing (NLP)

**Integrantes**:
- Antonio Lucas Michelon de Almeida
- Pedro Nery Affonso dos Santos
- Rafael Gordon Paves

In [1]:
import pandas as pd
import re
from dotenv import load_dotenv
import os
from google import genai
import json
import time
from tqdm import tqdm
from transformers import BertTokenizer, BertModel
import numpy as np
import torch
from models import FormatoResposta
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from agents import PlaylistGenerator, Validator
import joblib
import random

load_dotenv()

True

In [2]:
# Captura pedido de música
query = input("O que você gostaria de ouvir hoje? ")

## Playlist gerada a partir de uma única requisição ao Gemini (LLM)

In [10]:
def monta_playlist(texto):
    """
    Função que realiza uma requisição de montagem de playlist diretamente para o Gemini.
    """
    
    prompt = f"""songs_filtrado
    Você é um especialista em música e construção de playlists. 
    Você deve analisar o seguinte pedido de playlist e construir uma seleção de músicas que se encaixem no tema solicitado.
    Refira-se EXCLUSIVAMENTE ao pedido fornecido para criar a playlist.
    A playlist deve ser composta por 10 músicas, com o nome da música, do artista e possível featuring.
    A resposta deve ser uma lista em formato JSON, com os seguintes campos:
    - nome_musica: Nome da música
    - artista: Nome do artista
    - featuring: Nome do artista convidado (caso exista)

    Pedido do usuário: {texto}
    """

    client = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))
    resposta = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,
        config={
            "response_mime_type": "application/json",
            'response_schema': list[FormatoResposta],
            'temperature': 1.0
            # 'max_output_tokens': 500,
        }
    )
    return resposta

def playlist_gemini(query):
    # Resposta do Gemini
    resposta = monta_playlist(query)
    # Converte a resposta em um dicionário
    resposta_dict = json.loads(resposta.text)
    # Cria um DataFrame a partir da lista de dicionários
    df_one_shot = pd.DataFrame(resposta_dict)
    # Renomeia as colunas
    df_one_shot.rename(columns={'nome_musica': 'Nome da Música', 'artista': 'Artista', 'featuring': 'Featuring'}, inplace=True)
    return df_one_shot

# df_one_shot

## Playlist gerada a partir de interações entre agentes

In [18]:
def playlist_agentes(query):
    gerador = PlaylistGenerator()
    critico = Validator(query)

    # Envia ao gerador de Playlist o request do usuario
    playlist = gerador.initialRequest(query).text
    resposta_dict = json.loads(playlist)
    df_agentes = pd.DataFrame(resposta_dict)
    # display(df_agentes)
    # print("*"*150)

    repeticoes = 5
    for _ in range(repeticoes):
        feedback = critico.refactorRequest(playlist)   
        # print(feedback.text)
        # print("*"*150)
        # print(df_agentes.from_dict(json.load(feedback)))

        playlist = gerador.refactorPlaylist(feedback).text
        resposta_dict = json.loads(playlist)
        df_agentes = pd.DataFrame(resposta_dict)
        # display(df_agentes)
        # print("*"*150)

    return df_agentes

## Playlist gerada a partir de BERT - Criando os embeddings

In [ ]:
df = pd.read_csv('data/songs_filtrado.csv')
X = df["lyrics"]
y = df['tag']

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

# Utiliza a GPU para fazer as operações, fazendo muito mais rápido para modelos grandes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def get_embeddings(text, model, tokenizer):
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
        outputs = model(**inputs)
        cls_embedding = outputs.last_hidden_state[0, 0, :].cpu()

        # Para nao gastar tanta memoria
        del inputs, outputs
    return cls_embedding

batch_size = 500
for batch_i, inicio in enumerate(tqdm(range(0, len(X), batch_size))):
    # Se chegou ao fim, garante nao passar o index
    fim = inicio+batch_size if inicio+batch_size < len(X) else len(X)

    # Pega pequena amostra, para evitar estourar a memoria RAM (kk)
    batch = X.iloc[inicio:fim]

    embeddings = []
    for i in range(len(batch)):
        e = get_embeddings(batch.iloc[i], model, tokenizer)
        # print(f"{batch.iloc[i]} :{e.detach().numpy()}")
        embeddings.append(e.detach().numpy())
    
    embeddings = np.array(embeddings)

    # Salva os embeddings por batch, depois vamos juntar tudo
    np.save(f'embeddings/bert_emb_{batch_i}.npy', embeddings) 

    # Libera variaveis pesadas
    del embeddings
    # Por usar GPU
    torch.cuda.empty_cache()

100%|██████████| 575/575 [51:22<00:00,  5.36s/it]


In [14]:
# embeddings = np.array([])

for i in tqdm(range(len(os.listdir("./embeddings")))):
    f = f"./embeddings/bert_emb_{i}.npy"

    if i == 0:
        embeddings = np.load(f)
        continue

    embeddings_load = np.load(f)
    embeddings = np.append(embeddings, embeddings_load, axis=0)

np.save(f'bert_final.npy', embeddings)

del embeddings_load
del embeddings

100%|██████████| 575/575 [00:45<00:00, 12.74it/s]


In [6]:
df = pd.read_csv('songs_filtrado.csv')
y = df['tag']
embeddings = np.load("./bert_final.npy")
X_train, X_test, y_train, y_test = train_test_split(embeddings, y, test_size=0.2, random_state=42)
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

c:\Users\PEDRO\OneDrive\Documentos\Insper\2025.1\NLP\playlist-generator-nlp\env\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

     country       0.59      0.31      0.41      1024
        misc       0.83      0.65      0.73      1214
         pop       0.63      0.70      0.66     15513
         rap       0.87      0.93      0.90     27106
          rb       0.49      0.18      0.26      3169
        rock       0.63      0.59      0.61      9427

    accuracy                           0.75     57453
   macro avg       0.68      0.56      0.59     57453
weighted avg       0.74      0.75      0.74     57453



In [ ]:
# joblib.dump(clf, 'bert_logistic_model.joblib')

['bert_logistic_model.joblib']

## Rodando Playlist gerada pelo BERT

In [5]:
import joblib
import numpy as np
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

# 1) Carregar modelo e dados
clf        = joblib.load('bert_logistic_model.joblib')
embeddings = np.load('bert_final.npy')                # shape (N, D)
metadata   = pd.read_csv('data/songs_filtrado.csv')   # len(metadata) == N ?

# 1.1) Verifica alinhamento
if embeddings.shape[0] != len(metadata):
    raise ValueError(
        f"`embeddings` tem {embeddings.shape[0]} linhas, "
        f"mas `metadata` tem {len(metadata)} registros."
    )

# 2) Carregar BERT
tokenizer  = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')
device     = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bert_model.to(device)



BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [6]:
def get_embedding(text: str, model: BertModel, tokenizer: BertTokenizer) -> np.ndarray:
    """Retorna o vetor CLS de uma string via BERT."""
    with torch.no_grad():
        inputs  = tokenizer(
            text,
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=512
        ).to(device)
        outputs = model(**inputs)
        emb     = outputs.last_hidden_state[0, 0, :].cpu().numpy()
        del inputs, outputs
    return emb


def gerar_playlist_bert(query: str, top_n: int = 10) -> pd.DataFrame:
    """
    Gera uma playlist de `top_n` músicas com base na similaridade
    entre o embedding da query e os embeddings do dataset,
    dentro da mesma tag prevista.
    """
    # 3) Embedding e predição de tag
    emb_q = get_embedding(query, bert_model, tokenizer).reshape(1, -1)
    tag_prevista = clf.predict(emb_q)[0]
    # print(f"Tag prevista: {tag_prevista}")

    # 4) Filtra apenas os índices cujas músicas têm essa tag
    idxs = metadata.index[metadata['tag'] == tag_prevista].to_numpy()
    if len(idxs) == 0:
        raise ValueError(f"Nenhuma música encontrada com tag “{tag_prevista}”.")
    sub_embs     = embeddings[idxs]
    sub_meta     = metadata.loc[idxs]

    # 5) Calcula similaridade e seleciona top_n
    sims    = cosine_similarity(emb_q, sub_embs)[0]
    top_idxs = sims.argsort()[::-1][:top_n]

    # 6) Monta o DataFrame final
    playlist = sub_meta.iloc[top_idxs].copy()
    return playlist[['title', 'artist', 'tag']]


In [7]:
playlist = gerar_playlist_bert(query, top_n=10)
df_bert = pd.DataFrame(playlist)
# print("\nSua playlist:\n", playlist.to_string(index=False))

## Mostrando todas as playlists

In [20]:
query = input("O que você gostaria de ouvir hoje? ")

df_one_shot = playlist_gemini(query)
df_agentes = playlist_agentes(query)

playlist = gerar_playlist_bert(query, top_n=10)
df_bert = pd.DataFrame(playlist)

try:
    df_one_shot = df_one_shot.drop(columns=["Featuring"])
except:
    pass

try:
    df_agentes = df_agentes.drop(columns=["featuring"])
except:
    pass

try:
    df_bert = df_bert.drop(columns=["tag"]).reset_index(drop=True)
except:
    pass

dfs = [df_one_shot, df_agentes, df_bert]

choice = random.randint(0, 2)
display(dfs[choice])
dfs.pop(choice)
print("*"*150)
choice = random.randint(0, 1)
display(dfs[choice])
dfs.pop(choice)
print("*"*150)
display(dfs[0])
print("*"*150)

,nome_musica,artista
0,Down to River,Mitchell Dae
1,Riptide,Vance Joy
2,We're Going Home,Vance Joy
3,Hollow Lover,OWENN
4,Flowers in Your Hair,The Lumineers
5,Rivers and Roads,The Head and the Heart
6,Helplessly Hoping,"Crosby, Stills & Nash"
7,Skinny Love,Bon Iver
8,The Mariner's Revenge Song,The Decemberists
9,Tiger Striped Sky,Roo Panes


******************************************************************************************************************************************************


,title,artist
0,Mar Português,Fernando Pessoa
1,Pedras no Caminho,Augusto Cury
2,5 Momentos Genius: Espírito Vândalo do Funkero...,Genius Brasil
3,Os Melhores Álbuns de 2017 da Comunidade Genius,Genius Brasil
4,5 Momentos Genius: Castelos Ruínas um mar de ...,Genius Brasil
5,Os 10 Melhores Discos de Rap Nacional 2010-2014,Genius Brasil
6,Criolo - Convoque Seu Buda Análise,Genius Brasil
7,As 15 Melhores Lovesongs do Rap Nacional,Genius Brasil
8,Leia todas as letras de Heresia estreia solo d...,Djonga
9,Castelos Ruínas Tracklist e Capa,BK'


******************************************************************************************************************************************************


,Nome da Música,Artista
0,Down to River,Mitchel Dae
1,Riptide,Vance Joy
2,Little Talks,Of Monsters and Men
3,Rivers and Roads,The Head and the Heart
4,Holocene,Bon Iver
5,The Stable Song,Gregory Alan Isakov
6,Skinny Love,Birdy
7,We're Going Home,Vance Joy
8,The Boxer,Simon & Garfunkel
9,Hallelujah,Jeff Buckley


******************************************************************************************************************************************************
